In [11]:
import sys
sys.path.append("/home/charleshiggins/LazyMDP")
# training imports
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from wandb.integration.sb3 import WandbCallback
import wandb
import gymnasium as gym
# general imports
import argparse
import os
from lazywrapper import LazyWrapper, LastObservationWrapper

# define function
def make_custom_env(original_env, n_envs, seed, loaded_default_policy, penalty):
    def _init():
        env = LazyWrapper(LastObservationWrapper(gym.make(original_env)), default_policy=loaded_default_policy, penalty = penalty)
        return env
    return _init
# set parameters
default_model = PPO.load("/home/charleshiggins/LazyMDP/baselines/suboptimal_pretrained_policies/ppo_BreakoutNoFrameskip-v4_0")
seed = 0
penalty = -1
original_env_name = "BreakoutNoFrameskip-v4"
n_envs = 4
# make environment
vec_env = make_vec_env(env_id=make_custom_env(original_env_name, n_envs, seed, default_model, penalty), n_envs=n_envs)
vec_obs = vec_env.reset()
# make original environment
original_env = make_vec_env(original_env_name, n_envs=4)
obs = original_env.reset()

In [ ]:
original_env.step([0,1,0,1])

In [ ]:
vec_env.step([2,2,2,2])

In [13]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.atari_wrappers import AtariWrapper
# eval_env = Monitor(gym.make("Y"))
def make_atari_env(original_env):
    def _init():
        env = AtariWrapper(gym.make(original_env))
        return env
    return _init
env_eval = make_vec_env(env_id=make_atari_env("BreakoutNoFrameskip-v4"))
mean_reward, std_reward = evaluate_policy(default_model, env_eval, n_eval_episodes=10)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")



Mean reward: 7.40 +/- 6.53
